In [1]:
from pathlib import Path

import evaluate
from datasets import concatenate_datasets
from transformers import IntervalStrategy

from utils.dataset_utils import get_politicalness_datasets
from utils.model_utils import finetune_models

In [2]:
RESULT_SUBDIRECTORY_NAME = "politicalness"
GET_DATASETS = lambda: get_politicalness_datasets()
TRAIN_DATASET_SAMPLE_SIZE = 1_000
EVAL_DATASET_SAMPLE_SIZE = 100

train_datasets_separate = []
eval_datasets = []
for dataset in GET_DATASETS():
    eval_dataset = dataset.take_even_class_distribution_sample(EVAL_DATASET_SAMPLE_SIZE)
    eval_dataset = eval_dataset.transform_for_inference()

    # Remove the eval sample from the source dataframe.
    dataset.dataframe = dataset.dataframe.loc[~dataset.dataframe.index.isin(eval_dataset.dataframe.index)]

    train_dataset = dataset.take_even_class_distribution_sample(TRAIN_DATASET_SAMPLE_SIZE)
    train_dataset = train_dataset.transform_for_inference()

    train_datasets_separate.append(train_dataset.to_huggingface())
    eval_datasets.append(eval_dataset.to_huggingface())

train_datasets = [
    concatenate_datasets(
        list(filter(
            lambda dataset: dataset.info.dataset_name != left_out_dataset.info.dataset_name, train_datasets_separate
        )),
        info=left_out_dataset.info
    ) for left_out_dataset in train_datasets_separate
]

In [3]:
TRAINING_SEED = 37
DATA_SEED = 37
METRIC = evaluate.load("accuracy")
EVAL_STRATEGY = IntervalStrategy.EPOCH

finetune_models(
    Path("dataset_benchmark", "leave_one_out", RESULT_SUBDIRECTORY_NAME),
    train_datasets,
    eval_datasets,
    METRIC,
    EVAL_STRATEGY,
    TRAINING_SEED,
    DATA_SEED
)

fine-tuning launch/POLITICS into dataset_benchmark/leave_one_out/political_leaning:
  article_bias_prediction


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.199993,0.333333
2,No log,1.326698,0.333333
3,No log,1.377927,0.333333


  commoncrawl_news_articles


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

KeyboardInterrupt

